In [ ]:
import psycopg2 
import matplotlib.pyplot as plt 
import numpy as np 
from dotenv import load_dotenv 
import os 
from dotenv import load_dotenv 

# Загружаем переменные окружения из файла .env 

In [ ]:
load_dotenv() 

# Переменные окружения

In [ ]:
db_name = os.getenv("DB_NAME") 
db_user = os.getenv("DB_USER") 
db_password = os.getenv("DB_PASSWORD") 
db_host = os.getenv("DB_HOST") 
db_port = int(os.getenv("DB_PORT")) 

# Подключение к базе данных 

In [ ]:
conn = psycopg2.connect( 
    dbname=db_name,       # Имя базы данных 
    user=db_user,         # Имя пользователя 
    password=db_password, # Пароль 
    host=db_host,         # Хост (по умолчанию localhost) 
    port=db_port          # Порт (по умолчанию 5432) 
)
# Создание курсора для выполнения запросов 
cur = conn.cursor() 
start_t:str = "2025-02-11 10:06:00" 
end_t:str = "2025-02-11 22:55:00" 
# Выполнение SQL-запроса 
cur.execute(f"""WITH t1 AS (  
    SELECT * 
    FROM fssp_ms_gateway.remote_message 
    WHERE type = 'RESPONSE' 
      AND created > '{start_t}' 
      AND created < '{end_t}' 
), 
time_diffs AS ( 
    SELECT t1.oper_uid, 
           t1.type, 
           t2.created AS request_created, 
           t1.created AS response_created, 
           EXTRACT(EPOCH FROM (t1.created - t2.created)) / 60 AS delta_minutes 
    FROM t1 
    JOIN fssp_ms_gateway.remote_message t2 ON t1.oper_uid = t2.oper_uid 
    WHERE t2.type = 'REQUEST' 
) 
SELECT FLOOR(delta_minutes / 10) * 10 AS interval_start, 
       (FLOOR(delta_minutes / 10) + 1) * 10 AS interval_end, 
       COUNT(*) AS count_requests 
FROM time_diffs 
GROUP BY FLOOR(delta_minutes / 10) 
ORDER BY interval_start DESC;""") 

# Получение всех результатов 

In [ ]:
rows = cur.fetchall()

# Закрытие соединения с базой данных 

In [ ]:
cur.close() 
conn.close()

# Преобразуем в numpy для скорости

In [ ]:
array = np.array(rows)

# Строим график

In [ ]:
fig = plt.figure(dpi=400) 
 
# Изменяем её размеры 
fig.set_size_inches(24, 18) 
 
plt.bar([f"{x}_{y}" for x, y in zip(array[:, 0].astype(str), array[:, 1].astype(str))], array[:, 2]) 
# To show the plot 
# Задаем подписи для осей 
plt.ylabel('Кол-во сообщений в этом промежутке') 
plt.xlabel('Интервал') 
plt.xticks(rotation = 90) 
plt.show() 

In [ ]:
get_ipython().system('python -V')